## Random model tutorial


In this notebook, we present a more verbose version of the standard submission.py script, with the aim of explaining in detail how the main abstractions work and showing how easy it is to partecipate in the challenge. 

_NOTE_: this notebook is meant as a coding guide to the evaluation script, and a walk-through baseline submission to explain how to partecipate in the challenge. While you're free to experiment with this or other notebooks and even submit to the leaderboard from here, the _final_ submission should comply with the template scripts, as explained in the README.

Please contact the organizers on Slack should you have any doubt.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# check we are using the right interpreter with the right RecList version
!which python

/Users/jacopotagliabue/Documents/repos/evalRS-CIKM-2022/venv/bin/python


In [3]:
import os
import sys
sys.path.insert(0, '../')

_Basic imports, read the credentials from the env file_

In [4]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv

load_dotenv('../upload.env')

EMAIL = os.getenv('EMAIL')  # the e-mail you used to sign up
assert EMAIL != '' and EMAIL is not None
BUCKET_NAME = os.getenv('BUCKET_NAME') # you received it in your e-mail
PARTICIPANT_ID = os.getenv('PARTICIPANT_ID') # you received it in your e-mail
AWS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY') # you received it in your e-mail
AWS_SECRET_KEY = os.getenv('AWS_SECRET_KEY') # you received it in your e-mail
UPLOAD = bool(os.getenv('UPLOAD'))  # it's a boolean, True if you want to upload your submission
LIMIT = int(os.getenv('LIMIT'))  # limit the number of test cases, for quick tests / iterations. 0 for no limit

print("Submission will be uploaded: {}".format(UPLOAD))
if LIMIT > 0:
    print("WARNING: only {} test cases will be used".format(LIMIT))

Submission will be uploaded: True


_NOTE: as long as there is a limit specified, the runner won't upload results: make sure to have LIMIT=0 when you want to submit to the leaderboard!_

In [5]:
from evaluation.EvalRSRunner import EvalRSRunner
from evaluation.EvalRSRecList import EvalRSRecList

_Declare our model, in this case, a random generator: any model needs to include an implementation of "predict", taking user IDs as input and returning a DataFrame with predictions as output._

In [6]:
from reclist.abstractions import RecModel


class RandomModel(RecModel):
    
    def __init__(self, items: pd.DataFrame):
        super(RandomModel, self).__init__()
        self.items = items

    def predict(self, user_ids: pd.DataFrame, k=10) -> pd.DataFrame:
        """
        
        This function takes as input all the users that we want to predict the top-k items for, and 
        returns all the predicted songs.

        While in this example is just a random generator, the same logic in your implementation 
        would allow for batch predictions of all the target data points.
        
        """
        num_users = len(user_ids)
        pred = self.items.sample(n=k*num_users, replace=True).index.values
        pred = pred.reshape(num_users, k)
        pred = np.concatenate((user_ids[['user_id']].values, pred), axis=1)
        return pd.DataFrame(pred, columns=['user_id', *[str(i) for i in range(k)]]).set_index('user_id')


_We inherit from EvalRSRunner, and implement the required method, train_model: train_model encapsulate all your training logic, and should return any model class, wrapping predictions as shown above._

RandomEvalRSRunner is the Python object that will run the evaluation loop and provide utility functions to access data assets, tests, and upload results to the leaderboard.

In [7]:
class RandomEvalRSRunner(EvalRSRunner):
    def train_model(self, train_df: pd.DataFrame):
        """
        Implement here your training logic. Since our example method is a simple random model,
        we actually don't use any training data to build the model, but you should ;-)

        At the end of training, you should return a model class that implements the `predict` method,
        as RandomModel does.
        """
        return RandomModel(self.df_tracks)

_We initiliaze the runner with our credentials_

In [8]:
runner = RandomEvalRSRunner(
    num_folds=4,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    participant_id=PARTICIPANT_ID,
    bucket_name=BUCKET_NAME,
    email=EMAIL)

LFM dataset already downloaded. Skipping download.


_Let's inspect the main data assets first_

In [9]:
runner.df_tracks.head()

,track,artist_id,artist,albums_id,albums
track_id,,,,,
1,A Matter of Time,3,Foo Fighters,"[1, 605875, 67655, 2097]","[Wasting Light, Wasting Light Japan 1st Press ..."
2,Hangar 18,1,Megadeth,"[2113, 2, 83531, 17004, 11820, 379470, 440557,...","[Greatest Hits: Back to the Start, Rust In Pea..."
3,Up the Downstair,4,Porcupine Tree,"[3, 302086, 354478, 302545, 303154, 116, 84, 3...","[Spiral Circus Live, Stars Die: The Delerium Y..."
4,Mr. Carter (Featuring Jay-Z),2,Lil Wayne,[4],[Tha Carter 3]
5,Mixtaped,5,No-Man,[5],[Schoolyard Ghosts]


In [10]:
runner._get_train_set(3).head()

,user_id,artist_id,album_id,track_id,timestamp
78987589,17521972,3128,16794,21996,1332191855
98918751,46761744,13037,203461,314233,1325398060
13600966,5974006,10112,133620,639037,1269897479
6227197,2402401,40372,451280,607198,1299508575
15613678,35233465,3765,8082,370549,1386525644


In [11]:
runner.df_users.head()

,user_id,country,age,gender,playcount,registered_unixtime,novelty_artist_avg_month,novelty_artist_avg_6months,novelty_artist_avg_year,mainstreaminess_avg_month,...,relative_le_per_hour14,relative_le_per_hour15,relative_le_per_hour16,relative_le_per_hour17,relative_le_per_hour18,relative_le_per_hour19,relative_le_per_hour20,relative_le_per_hour21,relative_le_per_hour22,relative_le_per_hour23
0,384,UK,35,m,42139,1035849600,0.276629,0.044439,0.309429,0.024655,...,0.0279,0.0410,0.0811,0.1506,0.0377,0.0298,0.0132,0.0007,0.0001,0.0151
1,1206,None,-1,n,33103,1035849600,0.437224,0.109671,0.513787,0.181991,...,0.0016,0.0115,0.0276,0.0662,0.0790,0.0838,0.0995,0.1195,0.0968,0.0839
2,2622,None,-1,None,2030,1037404800,0.604828,0.043923,0.698983,0.052310,...,0.1320,0.0493,0.0317,0.0470,0.0658,0.1127,0.1112,0.0680,0.0437,0.0254
3,2732,None,-1,n,147,1037577600,0.756973,0.020071,0.882801,0.005092,...,0.0598,0.0726,0.0171,0.0342,0.0769,0.1453,0.0470,0.0513,0.0085,0.0128
4,3653,UK,31,m,18504,1041033600,0.380005,0.045207,0.424411,0.042821,...,0.0541,0.0518,0.0564,0.0554,0.0849,0.0954,0.0833,0.0657,0.0471,0.0595


_Finally, we run the evaluation code_

In [12]:
runner.evaluate(upload=UPLOAD, limit=LIMIT)


Performing Training for fold 1/4...
Performing Evaluation for fold 1/4...
============= TEST RESULTS ===============
Test Type        : stats
Test Description : 
Test Result      : {'num_users': 926, 'max_items': 5, 'min_items': 1}

============= TEST RESULTS ===============
Test Type        : HIT_RATE
Test Description : 
Test Result      : 0.0

Generating reports at 2022-07-24 15:42:30.370623

Performing Training for fold 2/4...
Performing Evaluation for fold 2/4...
============= TEST RESULTS ===============
Test Type        : stats
Test Description : 
Test Result      : {'num_users': 923, 'max_items': 3, 'min_items': 1}

============= TEST RESULTS ===============
Test Type        : HIT_RATE
Test Description : 
Test Result      : 0.0

Generating reports at 2022-07-24 15:42:53.390374

Performing Training for fold 3/4...
Performing Evaluation for fold 3/4...
============= TEST RESULTS ===============
Test Type        : stats
Test Description : 
Test Result      : {'num_users': 937, 'ma

### Customizing RecList

A huge motivation behind the Challenge is building as a community shareable insights in the form of working tests for our use case.

While your leaderboard score is ONLY influenced by the official tests as stated in the evaluation README, we encourage your final submissions to also include custom tests that you found helpful / insightful when improving your model.

The snippet below shows a working example of how to _extend_ the default RecList with additional tests, and run the evaluation code.

In [15]:
from reclist.abstractions import rec_test

class myRecList(EvalRSRecList):
    
    @rec_test(test_type='custom_test')
    def lucky_user_test(self):
        """
        Custom test, returning my lucky user from the catalog
        """
        from random import choice

        return {
          "luck_user": str(choice(self._x_test['user_id'].unique())) 
        }


_Re-run the evaluation with the additional test, which gets executed together with the default ones that produce the leaderboard score._

In [16]:
runner.evaluate(upload=UPLOAD, limit=LIMIT, custom_RecList=myRecList)


Performing Training for fold 1/4...
Performing Evaluation for fold 1/4...
============= TEST RESULTS ===============
Test Type        : custom_test
Test Description : Custom test, returning my lucky user from the catalog
Test Result      : {'luck_user': '36668664'}

============= TEST RESULTS ===============
Test Type        : stats
Test Description : 
Test Result      : {'num_users': 926, 'max_items': 5, 'min_items': 1}

============= TEST RESULTS ===============
Test Type        : HIT_RATE
Test Description : 
Test Result      : 0.0

Generating reports at 2022-07-24 15:45:17.131382

Performing Training for fold 2/4...
Performing Evaluation for fold 2/4...
============= TEST RESULTS ===============
Test Type        : custom_test
Test Description : Custom test, returning my lucky user from the catalog
Test Result      : {'luck_user': '16125937'}

============= TEST RESULTS ===============
Test Type        : stats
Test Description : 
Test Result      : {'num_users': 923, 'max_items': 3,

### Final submission to the committee

Since this is a code competition, you'll be required to submit your repository for statistical verification of your scores. 

Please consult the README carefully to make sure your project complies with the rules and follows the provided template script.